<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学術分野の分類（複数）
## jst_categorize_multi.ipynb
kakenhi_fine_tuning.ipynbでファインチューニングにより作成したモデルを用いて、新たな複数の学術文書（JSTやAMEDを含めた科研費に限らない）を11の大区分に分類するプログラム。  

### ライブラリのインストール
SageMakerでは初回だけで良い（Google colaboratoryでは毎回）  
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
# pipのアップデート（たまに走らせても良いかも）
!pip list
!python -m pip install --upgrade pip

In [ ]:
# ライブラリのインストール
!pip install torch
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

### Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# SageMakerでは不要
# from google.colab import drive
# drive.mount("/content/drive/")

## 複数の学術文書を分類
学習済み（ファインチューニング済み）のモデルを読み込み、複数の学術文書（実際には科研費概要テキスト）を分類する

### モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # SageMaker

loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
# loaded_model.cuda() # GPU対応
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

print("Finish")

### 分類精度検証用の科研費データ読み込み

In [ ]:
import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# open_original_csv = "KibanC_2021_Original.csv"
open_original_csv = "KibanC_2022-2022.csv" # 直近の１年（2022年）
# open_original_csv = "KibanC_2019-2020.csv" # 2018は「研究開始時の研究の概要」が無い
#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()


raw_data2

In [ ]:
import pandas as pd

# 科研費データベースからダウンロードした未加工のcsvファイルを指定
# open_original_csv = "KibanC_2021_Original.csv"
open_original_csv = "KibanC_2022-2022.csv" # 直近の１年（2022年）
# open_original_csv = "KibanC_2019-2020.csv" # 2018は「研究開始時の研究の概要」が無い
#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()

# 今後必要な行だけを取り出し、リネーム
kadai2 = raw_data2[["研究課題/領域番号", "審査区分", "研究開始時の研究の概要"]]
kadai2.columns = ["ID", "ShoKubun", "Abst"]

# 課題番号の重複を確認。課題番号でソートする。
kadai2["ID"].duplicated().any()
# kadai2 = kadai2.set_index("ID") # IDをインデックスに設定するコード
kadai2 = kadai2.sort_values("ID")

# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadai2))
print("概要が空白の課題数： %5d" % len(kadai2[kadai2["Abst"].isna()]))
kadai2 = kadai2.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d" % len(kadai2))

# Abst中の改行コードを削除
kadai2 = kadai2.replace('\r', '', regex=True)
kadai2 = kadai2.replace('\n', '', regex=True)

# Abstが英語のみの課題を削除
num_jpen = len(kadai2)
kadai2 = kadai2[kadai2["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadai2)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d" % num_jp)

# kadai.to_csv(data_path + "test1.csv", encoding = "cp932")

# 小区分が設定されていない課題を削除（旧分類、特設分野）
aaa = len(kadai2)
kadai2 = kadai2.dropna(subset=["ShoKubun"])
print("小区分がブランク： %5d" % (aaa - len(kadai2)))
print("小区分の設定あり： %5d" % len(kadai2))

# 小区分の文字列の数字部分だけを取り出す
kadai2["ShoKubun"] = kadai2["ShoKubun"].str[3:8]
kadai2 = kadai2.astype({"ShoKubun": int})

print("Finish")

### 大区分に変換

In [ ]:
#import pandas as pd
#from sklearn.model_selection import train_test_split

# 科研費の審査区分表データのcsvファイル
open_kubun_csv = "KubunTable.csv"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagamaker

# 審査区分テーブルのロード
kubun_table = pd.read_csv(data_path + open_kubun_csv, encoding="cp932")
kubun_table = kubun_table[["tabDai", "tabSho"]]

# 審査区分表の重複を削除（一つの小区分が２つまたは３つの『中区分』に所属することに由来する）
print("審査区分表の整理")
print("重複削除前の項目数： %3d" % len(kubun_table))
kubun_table = kubun_table.drop_duplicates()
print("重複削除後の項目数： %3d" % len(kubun_table))

# 大区分への変換
# mergeを用いて、審査区分表のデータと突合
print("統合前のデータ数： %5d" % len(kadai2))
kadaiDai2 = pd.merge(kadai2, kubun_table, left_on='ShoKubun', right_on='tabSho')
kadaiDai2 = kadaiDai2[["Abst", "tabDai", "ID", "ShoKubun"]]
print("統合したデータ数： %5d" % len(kadaiDai2))

kadaiDai2.info()

show_num = 0
kadaiDai2["Abst"][show_num]

print("Finish")

### 分類精度を複数ファイルで確認
３時間程度かかるかも  
sagemakerでは40分程度

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

results_binary = 'results'
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]


max_length = 512

num_data = len(kadaiDai2)
print("分類する課題数： %d" % num_data)
# num_data = 100
num_category = len(Daikubun)
# results = torch.zeros(num_data, num_category) # テンソルで変数を用意
results = np.zeros((num_data, num_category)) # メモリーが足りないと言われるのでテンソルではなくnumpy arrayにしてみた

for m in tqdm(range(num_data)):
  words = loaded_tokenizer.tokenize(kadaiDai2["Abst"][m])
  word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
  word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換
  
  y = loaded_model(word_tensor) # GPU未対応時の予測
  # y = loaded_model(word_tensor.cuda())  # GPU対応時の予測

  # results[m,:] = y[0]
  results[m,:] = y[0].detach().numpy() # テンソルをnumpy arrayに変換

# 変数をとりあえずバイナリで保存
np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存

print(results.shape)
results

### ここで一度保存
時間がかかる処理のあとなので、この段階で生成できたデータをファイルに保存

In [ ]:
# コードが不十分
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

results = np.load(data_path+results_binary+".npy")

# 変数をとりあえずバイナリで保存
#np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存
np.savez(data_path+"categolization_results", results, kadaiDai2, Daikubun)

分類精度を表示
混同行列

## 分類結果を表示

### 分類精度を複数ファイルで確認
confusion matrix （混同行列）を作成  
「マルチラベリング」で対応する方法もありそう

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

show_num = 0

results_binary = "results"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

# num_category = len(Daikubun)

results_2 = np.load(data_path+results_binary+".npy")

#results_3 = np.load(data_path+"categolization_results.npz", allow_pickle=True)
#results_2 = results_3['arr_0']
#kadaiDai2 = results_3['arr_1']
#Daikubun  = results_3['arr_2']

print("ロードした推定確率データのサイズ %d %d" % results_2.shape)

num_category = len(Daikubun)

results_2 = torch.tensor(results_2) # Softmax関数を使うためにテンソルに変換
m = torch.nn.Softmax(dim=1) # Softmax関数で確率に変換
results_2 = m(results_2)
results_2 = results_2.numpy() # numpy arrayに戻す

kadaiDai2['estimated'] = np.argmax(results_2, axis=1) # 最大要素のindexを返す（確率が最大の区分を返す）


print("重複課題の削除前 %5d" % len(kadaiDai2))
kadaiDai3 = kadaiDai2.drop_duplicates(subset='ID', keep=False)
print("重複課題の削除後 %5d" % len(kadaiDai3))

duplicated_data = kadaiDai3[kadaiDai3.duplicated(subset='ID', keep=False)]
unique_id = duplicated_data['ID'].drop_duplicates()


#
#for kk in unique_id:
#  dup_set = duplicated_data[duplicated_data['ID'] == kk]
#  cat_est = dup_set['estimated'].tolist()[0]
#  cat_real = dup_set['tabDai'].tolist()
#
#  if cat_est in cat_real:
#    aaa = cat_real.index(cat_est)
#    print(dup_set[:aaa])
#  else:
#    aaa = cat_est
#
#print(aaa)
    





# https://analysis-navi.com/?p=553
confmat = confusion_matrix(kadaiDai3['tabDai'], kadaiDai3['estimated']) # 混同行列の取得。(true, predicted)の順番
confmat = pd.DataFrame(confmat,columns=["pred_" + str(l) for l in Daikubun], index=["act_" + str(l) for l in Daikubun])
# print(cm)


#dup_idx_all = kadaiDai2.drop_duplicated(subset='ID',keep=False)

#duplicated_data_all = kadaiDai2[dup_idx_all]
#duplicated_data_all = duplicated_data_all[['ID', 'tabDai', 'estimated']]
#print(duplicated_data_all.shape)
#duplicated_data_all = duplicated_data_all.groupby('ID').count()

#dup_idx_first = kadaiDai2[kadaiDai2.duplicated(subset='ID', keep='first')]
#dup_idx_first = dup_idx_first.index

#for kk in dup_idx_first:
#  print(kk)


#duplicated_data.to_csv(data_path+"duplicated_mat.csv")

#kubun_estimated = np.argmax(results_2, axis=1)
#kubun_real      = kadaiDai2["tabDai"]
#
#kubun_results = pd.DataFrame(
#    data={
#        'estimated':kubun_estimated,
#        'real':kubun_real
#    }
#)

# 結果のグラフを表示
plt.bar(Daikubun, results_2[show_num,:])
print(kadaiDai3["Abst"][show_num])
print(kadaiDai3["ShoKubun"][show_num])
print(kadaiDai3['ID'][show_num])
print("実際　大区分" + Daikubun[kadaiDai3["tabDai"][show_num]])
print("推定　大区分" + Daikubun[kadaiDai3["estimated"][show_num]])
print(int(results_2[show_num, kadaiDai3["estimated"][show_num]]*100))
#kadaiDai2 = kadaiDai2[["Abst", "tabDai", "ID", "ShoKubun"]]

#out_mat = kubun_results.value_counts(sort=False)
#kubun_results.aggregate


#out_mat.to_csv(data_path+"result_mat.csv")

#dup_idx_first
confmat